# ETL Processes
Use this notebook to develop the ETL process for each of your tables before completing the `etl.py` file to load the whole datasets.

In [38]:
#import required libraries
import os
import glob
import psycopg2
import pandas as pd
from sql_queries import *

In [39]:
#connect to database
conn = psycopg2.connect("host=127.0.0.1 dbname=sparkifydb user=student password=student")
cur = conn.cursor()

In [40]:
def get_files(filepath):
    """fucntion to read all files from directory"""
    all_files = []
    for root, dirs, files in os.walk(filepath):
        files = glob.glob(os.path.join(root,'*.json'))
        for f in files :
            all_files.append(os.path.abspath(f))
    
    return all_files

# Process `song_data`
In this first part, you'll perform ETL on the first dataset, `song_data`, to create the `songs` and `artists` dimensional tables.

Let's perform ETL on a single song file and load a single record into each table to start.
- Use the `get_files` function provided above to get a list of all song JSON files in `data/song_data`
- Select the first song in this list
- Read the song file and view the data

In [4]:
song_files = get_files('data/song_data')

In [5]:
song_files

['/home/workspace/data/song_data/A/B/C/TRABCRU128F423F449.json',
 '/home/workspace/data/song_data/A/B/C/TRABCTK128F934B224.json',
 '/home/workspace/data/song_data/A/B/C/TRABCUQ128E0783E2B.json',
 '/home/workspace/data/song_data/A/B/C/TRABCIX128F4265903.json',
 '/home/workspace/data/song_data/A/B/C/TRABCYE128F934CE1D.json',
 '/home/workspace/data/song_data/A/B/C/TRABCEC128F426456E.json',
 '/home/workspace/data/song_data/A/B/C/TRABCXB128F4286BD3.json',
 '/home/workspace/data/song_data/A/B/C/TRABCAJ12903CDFCC2.json',
 '/home/workspace/data/song_data/A/B/C/TRABCPZ128F4275C32.json',
 '/home/workspace/data/song_data/A/B/C/TRABCKL128F423A778.json',
 '/home/workspace/data/song_data/A/B/C/TRABCEI128F424C983.json',
 '/home/workspace/data/song_data/A/B/C/TRABCFL128F149BB0D.json',
 '/home/workspace/data/song_data/A/B/B/TRABBBV128F42967D7.json',
 '/home/workspace/data/song_data/A/B/B/TRABBLU128F93349CF.json',
 '/home/workspace/data/song_data/A/B/B/TRABBXU128F92FEF48.json',
 '/home/workspace/data/so

In [6]:
filepath = 'data/song_data'

In [6]:
df = pd.read_json(song_files[5],lines=True)
df.head()

,artist_id,artist_latitude,artist_location,artist_longitude,artist_name,duration,num_songs,song_id,title,year
0,AR0IAWL1187B9A96D0,8.4177,Panama,-80.11278,Danilo Perez,197.19791,1,SONSKXP12A8C13A2C9,Native Soul,2003


In [8]:
#preview columns
df.columns

Index(['artist_id', 'artist_latitude', 'artist_location', 'artist_longitude',
       'artist_name', 'duration', 'num_songs', 'song_id', 'title', 'year'],
      dtype='object')

In [9]:
#Check datatypes
df.dtypes

artist_id            object
artist_latitude     float64
artist_location      object
artist_longitude    float64
artist_name          object
duration            float64
num_songs             int64
song_id              object
title                object
year                  int64
dtype: object

## #1: `artists` Table
#### Extract Data for Artists Table
- Select columns for artist ID, name, location, latitude, and longitude
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `artist_data`

In [7]:
#Select columns for artist table
artist_data = df[['artist_id','artist_name','artist_location','artist_longitude','artist_longitude']]
artist_data

,artist_id,artist_name,artist_location,artist_longitude,artist_longitude
0,AR0IAWL1187B9A96D0,Danilo Perez,Panama,-80.11278,-80.11278


In [8]:
#convert array to list
artist_data = list(df[['artist_id','artist_name','artist_location','artist_longitude','artist_longitude'\
                 ]].values[0])

In [9]:
#preview
artist_data

['AR0IAWL1187B9A96D0', 'Danilo Perez', 'Panama', -80.11278, -80.11278]

#### Insert Record into Artist Table
Implement the `artist_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song's artist into the `artists` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `artists` table in the sparkify database.

In [10]:
cur.execute(artist_table_insert, artist_data)
conn.commit()

Run `test.ipynb` to see if you've successfully added a record to this table.

## #2: `songs` Table
#### Extract Data for Songs Table
- Select columns for song ID, title, artist ID, year, and duration
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `song_data`

In [11]:
#select song_data columns
song_data = df[['song_id','title','artist_id','year','duration']]
song_data

,song_id,title,artist_id,year,duration
0,SONSKXP12A8C13A2C9,Native Soul,AR0IAWL1187B9A96D0,2003,197.19791


In [12]:
#Convert song data values from array to list
song_data = song_data.values.tolist()

In [13]:
#index to zero
song_data = song_data[0]

#### Insert Record into Song Table
Implement the `song_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song into the `songs` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songs` table in the sparkify database.

In [14]:
cur.execute(song_table_insert, song_data)
conn.commit()

Run `test.ipynb` to see if you've successfully added a record to this table.

# Process `log_data`
In this part, you'll perform ETL on the second dataset, `log_data`, to create the `time` and `users` dimensional tables, as well as the `songplays` fact table.

Let's perform ETL on a single log file and load a single record into each table.
- Use the `get_files` function provided above to get a list of all log JSON files in `data/log_data`
- Select the first log file in this list
- Read the log file and view the data

In [41]:
#call function and store returned value in log_files
log_files = get_files('data/log_data')

In [19]:
filepath = 'data/log_data'

In [42]:
# create dataframe from log_data using pandas
df_log = pd.read_json(log_files[0],lines=True)
#preview dataframe
df_log.head()

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,Sydney Youngblood,Logged In,Jacob,M,53,Klein,238.07955,paid,"Tampa-St. Petersburg-Clearwater, FL",PUT,NextSong,1.540558e+12,954,Ain't No Sunshine,200,1543449657796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",73
1,Gang Starr,Logged In,Layla,F,88,Griffin,151.92771,paid,"Lake Havasu City-Kingman, AZ",PUT,NextSong,1.541057e+12,984,My Advice 2 You (Explicit),200,1543449690796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",24
2,3OH!3,Logged In,Layla,F,89,Griffin,192.52200,paid,"Lake Havasu City-Kingman, AZ",PUT,NextSong,1.541057e+12,984,My First Kiss (Feat. Ke$ha) [Album Version],200,1543449841796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",24
3,RÃÂ¶yksopp,Logged In,Jacob,M,54,Klein,369.81506,paid,"Tampa-St. Petersburg-Clearwater, FL",PUT,NextSong,1.540558e+12,954,The Girl and The Robot,200,1543449895796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",73
4,Kajagoogoo,Logged In,Layla,F,90,Griffin,223.55546,paid,"Lake Havasu City-Kingman, AZ",PUT,NextSong,1.541057e+12,984,Too Shy,200,1543450033796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",24


## #3: `time` Table
#### Extract Data for Time Table
- Filter records by `NextSong` action
- Convert the `ts` timestamp column to datetime
  - Hint: the current timestamp is in milliseconds
- Extract the timestamp, hour, day, week of year, month, year, and weekday from the `ts` column and set `time_data` to a list containing these values in order
  - Hint: use pandas' [`dt` attribute](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.dt.html) to access easily datetimelike properties.
- Specify labels for these columns and set to `column_labels`
- Create a dataframe, `time_df,` containing the time data for this file by combining `column_labels` and `time_data` into a dictionary and converting this into a dataframe

In [44]:
#filter records by nextSong action
df_log = df_log[df_log.page == 'NextSong']
df_log.head()

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,Sydney Youngblood,Logged In,Jacob,M,53,Klein,238.07955,paid,"Tampa-St. Petersburg-Clearwater, FL",PUT,NextSong,1.540558e+12,954,Ain't No Sunshine,200,1543449657796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",73
1,Gang Starr,Logged In,Layla,F,88,Griffin,151.92771,paid,"Lake Havasu City-Kingman, AZ",PUT,NextSong,1.541057e+12,984,My Advice 2 You (Explicit),200,1543449690796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",24
2,3OH!3,Logged In,Layla,F,89,Griffin,192.52200,paid,"Lake Havasu City-Kingman, AZ",PUT,NextSong,1.541057e+12,984,My First Kiss (Feat. Ke$ha) [Album Version],200,1543449841796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",24
3,RÃÂ¶yksopp,Logged In,Jacob,M,54,Klein,369.81506,paid,"Tampa-St. Petersburg-Clearwater, FL",PUT,NextSong,1.540558e+12,954,The Girl and The Robot,200,1543449895796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",73
4,Kajagoogoo,Logged In,Layla,F,90,Griffin,223.55546,paid,"Lake Havasu City-Kingman, AZ",PUT,NextSong,1.541057e+12,984,Too Shy,200,1543450033796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",24


In [45]:
#convert timestamp to datetime 
t = pd.to_datetime(df_log['ts'], unit = 'ms')
t.head()

0   2018-11-29 00:00:57.796
1   2018-11-29 00:01:30.796
2   2018-11-29 00:04:01.796
3   2018-11-29 00:04:55.796
4   2018-11-29 00:07:13.796
Name: ts, dtype: datetime64[ns]

In [46]:
#create time_data, extract datetime values
time_data = (t.values.tolist()[0],t.dt.hour.values.tolist()[0],t.dt.day.values.tolist()[0],t.dt.weekofyear.values.tolist()[0],t.dt.month.values.tolist()[0]\
             ,t.dt.year.values.tolist()[0],t.dt.weekday.values.tolist()[0],df_log['sessionId'].values.tolist()[0])
column_labels = ('Start time','hour','day','week of year','month','year','weekday','sessionid')


In [26]:
df_log['ts'].values[0]

numpy.datetime64('2018-11-29T00:00:57.796000000')

In [47]:
#create pandas dataFrame
time_df = pd.DataFrame([time_data],columns=column_labels)
time_df.head()

,Start time,hour,day,week of year,month,year,weekday,sessionid
0,1543449657796000000,0,29,48,11,2018,3,954


In [51]:
#convert start time back to datetime
time_df['Start time'] = pd.to_datetime(time_df['Start time'])
time_df.head()

,Start time,hour,day,week of year,month,year,weekday,sessionid
0,2018-11-29 00:00:57.796,0,29,48,11,2018,3,954


#### Insert Records into Time Table
Implement the `time_table_insert` query in `sql_queries.py` and run the cell below to insert records for the timestamps in this log file into the `time` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `time` table in the sparkify database.

In [52]:
for i, row in time_df.iterrows():
    cur.execute(time_table_insert, list(row))
    conn.commit()

ProgrammingError: column "start_time" is of type bigint but expression is of type timestamp without time zone
LINE 1: ...ur,day,week,month,year,weekday,session_id) Values('2018-11-2...
                                                             ^
HINT:  You will need to rewrite or cast the expression.


Run `test.ipynb` to see if you've successfully added records to this table.

## #4: `users` Table
#### Extract Data for Users Table
- Select columns for user ID, first name, last name, gender and level and set to `user_df`

In [30]:
#select user table colums
user_df = df_log[['userId','firstName','lastName','gender','level','userAgent']]

#### Insert Records into Users Table
Implement the `user_table_insert` query in `sql_queries.py` and run the cell below to insert records for the users in this log file into the `users` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `users` table in the sparkify database.

In [31]:
for i, row in user_df.iterrows():
    cur.execute(user_table_insert, row)
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

## #5: `songplays` Table
#### Extract Data and Songplays Table
This one is a little more complicated since information from the songs table, artists table, and original log file are all needed for the `songplays` table. Since the log file does not specify an ID for either the song or the artist, you'll need to get the song ID and artist ID by querying the songs and artists tables to find matches based on song title, artist name, and song duration time.
- Implement the `song_select` query in `sql_queries.py` to find the song ID and artist ID based on the title, artist name, and duration of a song.
- Select the timestamp, user ID, level, song ID, artist ID, session ID, location, and user agent and set to `songplay_data`

#### Insert Records into Songplays Table
- Implement the `songplay_table_insert` query and run the cell below to insert records for the songplay actions in this log file into the `songplays` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songplays` table in the sparkify database.

In [32]:
for index, row in df_log.iterrows():

    # get songid and artistid from song and artist tables
    cur.execute(song_select, (row.song, row.artist,row.length))
    results = cur.fetchone()
    
    if results:
        songid, artistid = results
    else:
        songid, artistid = None, None

    # insert songplay record
    songplay_data = (row.ts,row.userId,row.level,songid,artistid,row.sessionId,row.location,row.userAgent)
    cur.execute(songplay_table_insert, songplay_data)
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

# Close Connection to Sparkify Database

In [53]:
conn.close()

# Implement `etl.py`
Use what you've completed in this notebook to implement `etl.py`.